In [1]:
import numpy as np
import random
from collections import Counter
from datetime import datetime
import keras
import sys
import io
import os
from pathlib import Path
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from datetime import datetime

Using TensorFlow backend.


In [2]:
with open('data/Mushrooms/Mushrooms.txt', 'r') as mushies:
    mushroom_accounts = mushies.read()

with open('data/Cannabis/Cannabis.txt', 'r') as pot:
    cannabis_accounts = pot.read()
    
with open('data/MDMA/MDMA.txt', 'r') as molly:
    mdma_accounts = molly.read()
    
with open('data/LSD/LSD.txt', 'r', encoding='utf-8') as acid:
    lsd_accounts = acid.read()

In [3]:
def sample(preds, temperature=1.0):
        """Helper function to sample an index from a probability array."""
        preds = np.asarray(preds).astype('float64')
        preds = np.exp(np.log(preds) / temperature)  # softmax
        preds = preds / np.sum(preds)                #
        probas = np.random.multinomial(1, preds, 1)  # sample index
        return np.argmax(probas)                     #


#returns a model trained on whatever text it is given. 
def make_model(text):
    #isolate characters unique to text file
    chars = sorted(list(set(text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    #define sequence length and step for iterating through text
    seqlen = 40
    step = seqlen
    sentences = []
    for i in range(0, len(text) - seqlen - 1, step):
        sentences.append(text[i: i + seqlen + 1])

    
    x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)

    #populate input matrix
    for i, sentence in enumerate(sentences):
        for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
            x[i, t, char_indices[char_in]] = 1
            y[i, t, char_indices[char_out]] = 1

    #generate model
    model = Sequential()
    model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
    model.add(Dense(len(chars), activation='softmax'))


    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.01),
        metrics=['categorical_crossentropy', 'accuracy']
    )


    def on_epoch_end(epoch, _):
        """Function invoked at end of each epoch. Prints generated text."""
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - seqlen - 1)

        diversity = 0.5
        
        print('----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model.fit(x, y,
        batch_size=128,
        epochs=50,
        callbacks=[print_callback])

    return model, x, y

In [79]:
mushrooms, mush_x, mush_y = make_model(mushroom_accounts[:len(mushroom_accounts)//2])

Epoch 1/50
62585/62585 [==============================] - 101s 2ms/step - loss: 1.9388 - categorical_crossentropy: 1.9388 - accuracy: 0.4498

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "belief. I began to cry because I was sca"
belief. I began to cry because I was scaring on the strenting and the trip I had to pain to be and being that had a self in the bot the strieg things was remember the sense of the all of the street the back to the bood and the upear and my moment and the most of the entile them in the down the face and looking the story meniar to the trip because I didnÂ’t know I had every had being which was and the trips in the was as spent into the s
Epoch 2/50
62585/62585 [==============================] - 98s 2ms/step - loss: 1.4747 - categorical_crossentropy: 1.4747 - accuracy: 0.5639

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "ities / Beings (37), Depression (15), Th"
ities / Beings (37

I think that I had been so as the trip to the movie was continued to take up to be forced out to do the path of a did the middle of the world of candate in the ground at the movie when I was a place, when I was amazed and the solve of exper
Epoch 24/50
62585/62585 [==============================] - 96s 2ms/step - loss: 1.2741 - categorical_crossentropy: 1.2741 - accuracy: 0.6144

----- Generating text after Epoch: 23
----- diversity: 0.5
----- Generating with seed: "und of it could make me panic. Since the"
und of it could make me panic. Since the trip was a good moment in the poor trip was some too the room and brought me a little band or she was really happy to the room and the moment of the last time I felt a little world. We were still probably the states of my room, she reminded me of reality and held the world and walk down to the day to walk out and passed on a heart on the darks and shower and a story, it was a skin and converse th
Epoch 25/50
62585/62585 [=====================

C:\Users\e_san_hjm\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


rip. I was a bad time. After a mild of my mind again.  I felt as if I was trying to be a good trips. It was a good orange change of truetion of th
Epoch 35/50
62585/62585 [==============================] - 98s 2ms/step - loss: 1.2651 - categorical_crossentropy: 1.2651 - accuracy: 0.6169

----- Generating text after Epoch: 34
----- diversity: 0.5
----- Generating with seed: "girlfriend and IÂ’m quite happy with my "
girlfriend and IÂ’m quite happy with my mind and set and donÂ’t like to be a visual power in the covers of the first carry and a couple of the trip for the blue doses with the couch in my mind while I thought that the mountains was a different bodies of the next day and I was a wild state which was both understanding on the bathroom and they were the stairs in the cabin white energy everything was handled the empty of the floor and pret
Epoch 36/50
62585/62585 [==============================] - 97s 2ms/step - loss: 1.2645 - categorical_crossentropy: 1.2645 - accuracy: 0.6169

In [82]:
mdma, mdma_x, mdma_y = make_model(mdma_accounts)

Epoch 1/50
41238/41238 [==============================] - 70s 2ms/step - loss: 2.1969 - categorical_crossentropy: 2.1969 - accuracy: 0.3944

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "an actually be in the experience from my"
an actually be in the experience from my friends to feel to the tooked to a feeling to the seding to standed to started to did to got have the drug the lofe into the gooded to get the mach to the had into have not experiences and the expriences (22), General (1)





































BODY WEIGHT:
120 lb








































BODY WEIGHT:
120 lb
































BODY WEIGHT:
12. me









Epoch 2/50
41238/41238 [==============================] - 73s 2ms/step - loss: 1.5667 - categorical_crossentropy: 1.5667 - accuracy: 0.5439

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "bed.

At this point I started calm down,"
bed.

At this point

t (or at least not act freely in a Â“high and my room and then my friend as I wasn't a depression I have to go home and had a good and I felt that I had seen to take an increase we were says the experience was a sensation of my friend had never say that I was all the blinked and have been tired and it was all day before and the moment of the pill as I never felt a bit thing to have posed to get the past and down a pill forever feel in a
Epoch 24/50
41238/41238 [==============================] - 74s 2ms/step - loss: 1.2657 - categorical_crossentropy: 1.2657 - accuracy: 0.6203

----- Generating text after Epoch: 23
----- diversity: 0.5
----- Generating with seed: " side of the road. She began speaking re"
 side of the road. She began speaking ready to say that pressed and didn't have been together. I was some time. I felt pretty better that I was the stroched the party and while to drink some Most thoughts with the more pills at a lot of and kinda it was interlets while almost a friend a

our neither of us were affected so she felt an anyone had some of the morning even feeling about my experience. I started to take the resolved and want to take the whole down reality of the come during my friend of day of MDMA and say to me. I was so much stambed that I had started to say, as well, the best time the house and the first time and when we were the way my pupils were the possibility of the compulsive that I was a little con
Epoch 46/50
41238/41238 [==============================] - 74s 2ms/step - loss: 1.2456 - categorical_crossentropy: 1.2456 - accuracy: 0.6253

----- Generating text after Epoch: 45
----- diversity: 0.5
----- Generating with seed: "'t have the strength to be up any longer"
't have the strength to be up any longer and started more a little more so I did found the sensation of experience I was something I had ever felt like a still in my body some more in the back of problems that they were a party in my life. I was so so I did not realized that I was feeli

In [86]:
lsd, lsd_x, lsd_y = make_model(lsd_accounts[:len(lsd_accounts)//2])

Epoch 1/50
49097/49097 [==============================] - 130s 3ms/step - loss: 2.0231 - categorical_crossentropy: 2.0231 - accuracy: 0.4275

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "e right after my trip. I didn't write it"
e right after my trip. I didn't write it hit over the real and there was a started to get I was the me we wore the end about the the to my friend and for a start the dood. I was really to the came and though in the table and I had a hate this store it was in the mind of while the so I was was a some to the though the condrance and and the strooms so I was a thought the some from the more this though the bedy and a trip was its we always
Epoch 2/50
49097/49097 [==============================] - 128s 3ms/step - loss: 1.5322 - categorical_crossentropy: 1.5322 - accuracy: 0.5480

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: " before.  During the day I experienced n"
 before.  During 

nt settle so I propose: 'He clicked his too man with a while driving to be in the point that the matter of a friend of people were seen a more substance through the woods of thoughts. A shit some of a bit of the trip with a came back to the mightion and started to go from the world with the cat we were still still still also all I was going to save many of the acid I was so we were all the experience with place with the mind, and I was
Epoch 24/50
49097/49097 [==============================] - 130s 3ms/step - loss: 1.2955 - categorical_crossentropy: 1.2955 - accuracy: 0.6082

----- Generating text after Epoch: 23
----- diversity: 0.5
----- Generating with seed: "t acid I had ever had (and I was VERY ex"
t acid I had ever had (and I was VERY experiments and perceived of the way to the state of the room and the morning in some street and we were experimenting a spiritual present to a few more while we sat on a seconds of the room and I would have been about how the fear pulled outside d

med to be somewhere in the stratosphere. We took a regulation. It was been a child to me, so I perceptions that I was still a thoughts and was a little bit of the two trip to my friend and everything was like the entire constantly faces in the party and been sending the mind was still seen and I didn't have to control the time I started to feel my face to a scenes of the constantly still had starting to see the stuff and saying experien
Epoch 46/50
49097/49097 [==============================] - 128s 3ms/step - loss: 1.2791 - categorical_crossentropy: 1.2791 - accuracy: 0.6128

----- Generating text after Epoch: 45
----- diversity: 0.5
----- Generating with seed: "eb. I think the web represented many per"
eb. I think the web represented many personal thing was an either and still beautiful.  I didnt explain this time in the couch and clarity of the way to the sun was trying to go to the computer mental remained and I sat down and coming to her back to the bed and not the bottom of the 

In [88]:
cannabis, cannabis_x, cannabis_y = make_model(cannabis_accounts)

Epoch 1/50
50814/50814 [==============================] - 175s 3ms/step - loss: 2.0563 - categorical_crossentropy: 2.0563 - accuracy: 0.4240

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: ", shapes, etc. I start to think differen"
, shapes, etc. I start to think different and I was the same for and I was down oftered on the facal to this point and the batto a do waster had some better on the strippy muth me to drang drance. more place of the spones of higher to the not experience: Not GivenÂ 
Published: Not GivenÂ 
Published: Mevead
Cannabis
(plant materience: Not GivenÂ 
Published: Mal 18 20 Diggraing) ] [ View as LaTeX (for geeks) ]
[ Switch Colors ]

Cannabis 
Epoch 2/50
50814/50814 [==============================] - 190s 4ms/step - loss: 1.4862 - categorical_crossentropy: 1.4862 - accuracy: 0.5625

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "know anything about it and I planned on "
know anything abo

as always high she immediantly knew I was half the world and the same time I was still to the street. I think I was so an altered my body and I was still don't know what I was a beautiful and I had a bright of the bed and the guys was a strange since and asked us and something from the park the state of second stuff. I was so far and should have been so I would be the process when the first time when I was probably the stoned of the cou
Epoch 24/50
50814/50814 [==============================] - 186s 4ms/step - loss: 1.2492 - categorical_crossentropy: 1.2492 - accuracy: 0.6236

----- Generating text after Epoch: 23
----- diversity: 0.5
----- Generating with seed: ". 

I smoked in the bathroom, with the s"
. 

I smoked in the bathroom, with the sustents of my head.

I can see what I was passing the street of ass the weed and don't know what I was the third and my friends were the sort of the side of the tables. I thought they were the action of my hands on the same shadowic sincent cares

C:\Users\e_san_hjm\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


7)




Â 




DOSE:
Â  
smoked
Cannabis
(plant material)





BODY WEIGHT:
125 lb





I could like to have a all of him eyes of the school of many times and realized that I was so stoned.  I can explore out of a comes of a time. I can remember the strong trip for a more of the body for me.  I was still because of him
Epoch 32/50
50814/50814 [==============================] - 175s 3ms/step - loss: 1.2417 - categorical_crossentropy: 1.2417 - accuracy: 0.6253

----- Generating text after Epoch: 31
----- diversity: 0.5
----- Generating with seed: "he drug take over, such as in the shower"
he drug take over, such as in the shower of the circles of a shood and watched the person was all of the couch and sat down the couch. The bottom for my eyes and then seemed to be a part of my eyes, and we were all of the street of my body and I had to breathe and somewhat the time that I had a thing to do not be a small the thought and I have not really got a couple of hit. There was a few minutes and t

In [4]:
# evaluate trained model, to check saved and retreived model later
def eval_model(model, x, y):
    print(model.evaluate(x[-2000:], y[-2000:], verbose=0))

In [5]:
def save_trained_model(trained_model, drug_name):
    now = datetime.now()
    model_json = trained_model.to_json()
    filename = drug_name + "_" + now.strftime("%m-%d_%H-%M-%S")
    with open("saved_models/" + filename + ".json", "w+") as json_file:
        json_file.write(model_json)
    # serialize weights
    trained_model.save_weights("saved_models/" + filename + ".h5")
    print(drug_name + "_" + now.strftime("%m-%d_%H-%M-%S") + " model saved!")

In [6]:
def load_trained_model(filename):
    # load model
    json_file = open("saved_models/" + filename + ".json", "r")
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = keras.models.model_from_json(loaded_model_json)
    # load weights
    loaded_model.load_weights("saved_models/" + filename + ".h5")
    print(filename + " model loaded")
    # compile
    loaded_model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.1),
        metrics=['categorical_crossentropy', 'accuracy']
    )
    return loaded_model

In [16]:
# Load saved model for proofing
# hardcode filename here
mdma = load_trained_model("mdma_12-01_15-09-43")
lsd = load_trained_model('lsd_12-01_18-13-19')
mushrooms = load_trained_model('mushrooms_11-30_16-56-31')
cannabis = load_trained_model('cannabis_12-02_02-52-16')

mdma_12-01_15-09-43 model loaded
lsd_12-01_18-13-19 model loaded
mushrooms_11-30_16-56-31 model loaded
cannabis_12-02_02-52-16 model loaded


In [8]:
def generate_text(model, accounts, seed, length):
    chars = sorted(list(set(accounts)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    seqlen = 40

    generated = ''
    sentence = seed
    print('----- Generating with seed: "' + sentence + '"')

    for i in range(length):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], 0.5)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char 
        generated += next_char
    return generated

In [18]:
# input a string seed of exactly 40 characters
print(generate_text(mdma, mdma_accounts, "This trip started out fantastic, but the", 4000))

----- Generating with seed: "This trip started out fantastic, but the"
y were all was take having the realizations of my friends was to find a different death to people at a time.  The feeling that I was something the drug was doing that I was so real effects and they wandered the people was a seriously had to go to the source your body and some of the sense at a really well, so I just had hope the more.  It took the music to me.  They were a dream and people was still all the hand to come up a lot of an experience with the sense of persons and have been to the potentially while my head out to the feeling and have a next of our perspective and had a week, relaxed of my friends were strange way.  I had a few moment of the day, and I was so real strungly things were in the music of my bed.  The first time I was sitting on this point I was a hallucination and had the arms who had a hales and they were so positive than anyway, and I was in a perspectives and a half to the house and showers

In [19]:
generate_text(lsd, lsd_accounts[:len(lsd_accounts)//2], "This trip started out fantastic, but the", 4000)

----- Generating with seed: "This trip started out fantastic, but the"


' strange that I seemed to be a consciousness into the end of the night were just pretended in my confusion on the really hard the state of so for a while that the pressure of my life that we were already only the sidewalk back into the parking under my strange of experiences of the person and walked to his aspects of different concept of the floor convinced and started to feel any black patterns or something like this was every dimension for a while, and then been a perfect control of in the other faces in the first a minute and my mind, and seemed to be a car in the surfected in the surface, and was a little bright and seemed to go to complex, I was not actually decided to get out of the drug and realized the third and got along around this point that he was still so strong and with a single relaxing and as problem, and I could do me to have some point in my best placed the first hour or so we can do and get the room, I could feel it was a level is a girl I was beginning to come to a